<b>Hypothesis and Inference</b>

In [1]:
from typing import Tuple
import math
def normal_approximation_to_binomial(n: int, p:float) -> Tuple[float,float]:
    mu = p * n
    sigma = math.sqrt(p*(1-p)*n)
    return mu, sigma

In [2]:
def normal_cdf(x: float, mu: float = 0, sigma: float = 1) -> float:
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

In [3]:
normal_probability_below = normal_cdf
def normal_probability_above(lo:float,
                             mu: float=0,
                             sigma: float=1) -> float:
    return 1 - normal_cdf(lo,mu,sigma)

def normal_probability_between(lo:float,
                               hi: float,
                               mu: float=0,
                             sigma: float=1) -> float:
    return normal_cdf(hi,mu,sigma) - normal_cdf(lo,mu,sigma)

def normal_probability_outside(lo:float,
                               hi: float, 
                               mu: float=0,
                             sigma: float=1) -> float:
    return 1 - normal_probability_between(lo,hi,mu,sigma)

In [4]:
def inverse_normal_cdf(p: float,
                       mu: float = 0,
                       sigma: float = 1,
                       tolerance: float = 0.00001) -> float:

    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)

    low_z = -10.0                      
    hi_z  =  10.0                      
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2     
        mid_p = normal_cdf(mid_z)      
        if mid_p < p:
            low_z = mid_z              
        else:
            hi_z = mid_z               

    return mid_z

In [5]:
def normal_upper_bound(proability:float,
                     mu: float=0,
                      sigma: float=1) -> float:
    return inverse_normal_cdf(proability,mu,sigma)


def normal_lower_bound(proability:float,
                     mu: float=0,
                      sigma: float=1) -> float:
    return inverse_normal_cdf(1- proability,mu,sigma)


def normal_two_sided_bounds(proability:float,
                           mu:float=0,
                           sigma: float = 1) -> Tuple[float,float]:
    tail_probability = (1-proability) / 2
    upper_bound = normal_upper_bound(tail_probability,mu,sigma)
    lower_bound = normal_lower_bound(tail_probability,mu,sigma)
    return upper_bound,lower_bound

In [6]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000,0.5)
print(mu_0, "   ",sigma_0)

500.0     15.811388300841896


In [7]:
lower_bound, upper_bound = normal_two_sided_bounds(0.95, mu_0, sigma_0)
print(lower_bound,"   ",upper_bound)

469.01026640487555     530.9897335951244


In [8]:
lo,hi = normal_two_sided_bounds(0.95,mu_0,sigma_0)
print(lo,"   ",hi)

469.01026640487555     530.9897335951244


In [9]:
mu_1,sigma_1 = normal_approximation_to_binomial(1000,0.55)
print(mu_1,"   ",sigma_1)

550.0     15.732132722552274


In [10]:
type_2_probability = normal_probability_between(lo,hi,mu_1,sigma_1)
power = 1 - type_2_probability
print(power)

0.8865480012953671


In [11]:
hi = normal_upper_bound(0.95,mu_0,sigma_0)
print(hi)

526.0073585242053


In [12]:
type_2_probability = normal_probability_below(hi,mu_1,sigma_1)
power = 1 -  type_2_probability
print(power)

0.9363794803307173


<b>p-Values</b>

In [16]:
def two_sided_p_values(x:float,mu:float=0,sigma:float=1)->float:
    if x>=mu:
        return 2*normal_probability_above(x,mu,sigma)
    else:
        return 2 * normal_probability_below(x,mu,sigma)
    

In [17]:
two_sided_p_values(529.5,mu_0,sigma_0)

0.06207721579598835

In [19]:
import random
exterme_value_count = 0
for _ in range(1000):
    num_heads = sum(1 if random.random()<0.5 else 0
                   for _ in range(1000))
    if num_heads >= 530 or num_heads <= 470:
        exterme_value_count += 1
        
    

In [20]:
two_sided_p_values(531.5,mu_0,sigma_0)

0.046345287837786575

In [21]:
upper_p_value = normal_probability_above
lower_p_value = normal_probability_below

In [22]:
print(upper_p_value(524.5, mu_0, sigma_0))
print(upper_p_value(526.5, mu_0, sigma_0))

0.06062885772582072
0.04686839508859242


<b>Confidence Intervals</b>

In [25]:
#using estmated value
p_hat = 525/1000
mu = p_hat
sigma = math.sqrt(p_hat*(1-p_hat)/1000)
print(sigma)

0.015791611697353755


In [26]:
normal_two_sided_bounds(0.95,mu,sigma)

(0.4940490278129096, 0.5559509721870904)

In [28]:
p_hat = 540 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000) 
print(sigma)
print(normal_two_sided_bounds(0.95, mu, sigma) )

0.015760710643876435
(0.5091095927295919, 0.5708904072704082)


<b>p-Hacking</b>

In [31]:
from typing import List

def run_experiment() -> List[bool]:
    return [random.random() < 0.5 for _ in range(1000)]

def reject_fairness(experiment: List[bool]) -> bool:
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

random.seed(0)
experiments = [run_experiment() for _ in range(1000)]
num_rejections = len([experiment
                        for experiment in experiments
                        if reject_fairness(experiment)])

<b>A small exercise on ad clicking by users</b>

In [32]:
def estimated_parameters(N: int, n: int) -> Tuple[float, float]:
    p = n / N
    sigma = math.sqrt(p * (1 - p) / N)
    return p, sigma

In [33]:
def a_b_test_statistic(N_A: int, n_A: int, N_B: int, n_B: int) -> float:
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

In [36]:
z = a_b_test_statistic(1000, 200, 1000, 180)
print(z)
print(two_sided_p_values(z))

-1.1403464899034472
0.254141976542236


In [38]:
z = a_b_test_statistic(1000, 200, 1000, 150)
print(z)
print(two_sided_p_values(z))

-2.948839123097944
0.003189699706216853


<b>Bayesian Inference</b>

In [40]:
def B(alpha: float, beta: float) -> float:
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

def beta_pdf(x: float, alpha: float, beta: float) -> float:
    if x <= 0 or x >= 1:
        return 0
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)